Reservoirs store water between timesteps.  They are nodes in the water network, and receive upstream flows and precipitation, and provide withdrawals to canals.  However, their outflows are determined by an optimization.

# Selecting Reservoirs

The reservoirs that should be modeled as storing water across period, rather than acting like stream nodes, differ depending on the timestep.  The function that provides a list of reservoirs is `getreservoirs(config)`.

In [1]:
include("../src/lib/readconfig.jl")
include("../src/lib/reservoirs.jl")

config = readconfig("../configs/standard.yml")

Dict{Any,Any} with 6 entries:
  "startweather" => 612
  "timestep"     => 6
  "netset"       => "usa"
  "filterstate"  => nothing
  "endmonth"     => "9/2010"
  "startmonth"   => "10/2000"

In [2]:
@doc getreservoirs

Return a DataFrame containing `collection` and `colid` fields matching those in the Water Network.

Any additional columns can be provided, to be used by other components.

Rows may be excluded, to represent that a given reservoir should be modeled as a stream at the specified timestep (in months).


In [3]:
getreservoirs(config)

,collection,colid,area,lat,lon,elev,MAXCAP
1,reservoir,1,NA,48.8733,-122.688,NA,5600
2,reservoir,2,145.03944,48.7583,-122.422,NA,77000
3,reservoir,3,556.84785,48.6583,-121.687,NA,177184
4,reservoir,4,556.84785,48.6483,-121.69,NA,316100
5,reservoir,5,2587.40001,48.7317,-121.067,NA,1633400
6,reservoir,6,2913.73875,48.7133,-121.13,NA,91600
7,reservoir,7,3001.79841,48.6983,-121.207,NA,10780
8,reservoir,8,769.22703,48.5483,-121.74,NA,161470
9,reservoir,9,NA,48.9317,-119.418,NA,40000
10,reservoir,10,797.71692,48.095,-123.555,NA,8100


# Optimization example

To understand the functioning of reservoirs, consider how they work in a very simple three gauge example.  The three guages example has three counties, with a river running through them.  The middle county has a reservoir.  Water is supplied only upstream (in counties 1 and 2) and consumed only downstream (in counties 2 and 3).

![Three counties diagram](https://raw.githubusercontent.com/AmericasWater/operational-problem/master/docs/three.png)

Note that the reservoir appears to be outside of the river system.  While it is spatially synonymous with the middle gauge, reservoirs get all of their inflows from "captured" water.  Any water that is not captured is allowed to run through the reservoir just like a stream.  Reservoir captures can also be negative, providing releases.

The `optimize-surface.jl` script models the constraints to satisfy surface water demands, using reservoirs to store water between periods.

In the three counties example, the first period has more rainfall than the second period, so that storage is optimal.

In [1]:
include("../src/optimize-surface.jl")

Loading from saved region network...


Loading from saved water network...
Optimize a model with 22 rows, 20 columns and 50 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-02, 1e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 1e+09]
Presolve removed 17 rows and 12 columns
Presolve time: 0.27s
Presolved: 5 rows, 8 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e-02   3.000000e+00   0.000000e+00      0s
       3    4.0000000e-02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.39 seconds
Optimal objective  4.000000000e-02
  5.969384 seconds (3.02 M allocations: 120.035 MB, 2.18% gc time)
waterfromsupersource
All zero.
withdrawals
[0.0,1.0,1.0,0.0,1.0,1.0]
Sum: 4.0
returns
All zero.
captures
[1.0,-1.0]
Sum: 0.0
Ignore:



48

First, look at the runoff values, with rows for the three gauges and columns for the two time periods.

In [2]:
runoff

3x2 Array{Float64,2}:
 2.0  1.0
 1.0  0.0
 0.0  0.0

The requirements are for one unit of water each period for each of the lower two gauges.

In [5]:
reshape(constraintoffset_allocation_recordedbalance(m).f, 3, 2)

3x2 Array{Float64,2}:
 0.0  0.0
 1.0  1.0
 1.0  1.0

The order of the parameters in the LP problem is:

In [6]:
parameters

4-element Array{Symbol,1}:
 :waterfromsupersource
 :withdrawals         
 :returns             
 :captures            

And the order of the constraint variables is:

In [8]:
constraints

5-element Array{Symbol,1}:
 :outflows     
 :balance      
 :returnbalance
 :storagemin   
 :storagemax   

Consider the constraint matrix one parameter at a time.  The first parameter is the water drawn from the supersource, which only affects the second constraint, `:balance`, the difference between water demand and water supply.  The objective function is such that supersource withdrawals are avoided.

In [3]:
full(house.A)[:, 1:6]

22x6 Array{Float64,2}:
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0

Normal withdrawals affect `:balance` as well, but they also affect the level of streamflow, which is constrained to be above 0.  See the top six rows in two blocks, in the upper left and lower right, corresponding to the two periods.  The gauges are ordered, by chance, such that the most upstream gauge is second, so a withdrawal from that gauge causes all three gauges to inch closer to empty.

In [4]:
full(house.A)[:, 7:12]

22x6 Array{Float64,2}:
  0.0   1.0   0.0   0.0   0.0   0.0
  1.0   1.0   0.0   0.0   0.0   0.0
  1.0   1.0   1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0   0.0
  0.0   0.0   0.0   1.0   1.0   0.0
  0.0   0.0   0.0   1.0   1.0   1.0
 -1.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0

We'll ignore the `:returns` constraint.  The last is the reservoir in the two periods.  Reservoir captures have the same affect on streamflows, at least downstream.  They have no direct effect on `:balance`.  Finally, they affect the last two constraints on the upper and lower bounds of the reservoir, where capture (or release) in period 1 affects both periods, while capture (or release) in period 2 only affects one.

In [6]:
full(house.A)[:, end-1:end]

22x2 Array{Float64,2}:
  0.0   0.0
  0.0   0.0
  1.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   1.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.0
 -1.0   0.0
 -1.0  -1.0
  1.0   0.0
  1.0   1.0

The result is that it is optimal to store one unit of water and release it in the second period.

In [7]:
constraining(house, sol.sol)

Ignore:



,solution,component,parameter,abovefail,belowfail
1,0.0,Allocation,waterfromsupersource,,balance.1
2,0.0,Allocation,waterfromsupersource,,balance.2
3,0.0,Allocation,waterfromsupersource,,balance.3
4,0.0,Allocation,waterfromsupersource,,balance.4
5,0.0,Allocation,waterfromsupersource,,balance.5
6,0.0,Allocation,waterfromsupersource,,balance.6
7,0.0,Allocation,withdrawals,outflows.3,balance.1
8,1.0,Allocation,withdrawals,outflows.3,balance.2
9,1.0,Allocation,withdrawals,outflows.3,balance.3
10,0.0,Allocation,withdrawals,"outflows.5, outflows.6",balance.4


Future work:
- The optimization does not include evaporation, since this would make the numbers for this toy example inconvenient.  Evaporation needs to be added to both the optimization and the example.

# Reservoir data

### Maximum Reservoir Storage (or Capacity)

Received Maximum capacity for 2641 reservoirs from the US Army Corps of Engineers. However, in the allreservoirs.csv file in the folder awash/data/reservoirs/,there are 2671 reservoirs in total. 

The allreservoir.csv database is being used in the reservoir optimization (reservoir.jl). The remaining 30 reservoirs are from the USGS website. 

Since we are unable to currently find the maximum capacity for those 30 reservoirs (reservoir codes mentioned below), for this example, the maximum capacity is taken as 0.

usgsres	3170500
usgsres	5113750
usgsres	6886900
usgsres	7233550
usgsres	7332610
usgsres	7342495
usgsres	7343460
usgsres	7344488
usgsres	8025350
usgsres	8051100
usgsres	8064550
usgsres	8072500
usgsres	8073000
usgsres	8079700
usgsres	8092500
usgsres	8095550
usgsres	8118000
usgsres	8123000
usgsres	8123755
usgsres	8123950
usgsres	8131200
usgsres	8134500
usgsres	8136600
usgsres	8180010
usgsres	8202800
usgsres	9125800
usgsres	9129550
usgsres	9143600
usgsres	9147022
usgsres	15225990


### Minimum reservoir storage

The minimum reservoir storage is assumed to be 10% of the maximum reservoir storage, i.e.

storagecapacitymin = 0.1 * storagecapacitymax

This data needs improvement -- we need to find the minimum storage for each reservoir rather than taking one assumptions for all reservoirs

### Initial Storage

The initial storage is assumed to be the average of minimum and maximum capacity, i.e.

Initial storage = (storagecapacitymax-storagecapacitymin)/2

This data needs improvement -- we need to find the initial storage of each reservoir

### Evaporation

Evaporation is currently assumed to be 1% of the storage capacity, i.e.

Evaporation = 0.01 * storagecapacitymax

The data needs improvement -- we need to find a linear relationship between evaporation and reservoir storage.